In [1]:
import numpy as np
import pandas as pd

In [2]:
def data_generate(size = 10000,leng = 11):
    data = np.array([chr(i) for i in range(65,65 + leng -1)])
    number = np.random.randint(10, size = (size,leng))
    count = 0
    path = np.array("←".join(data)).repeat(size)
    user = np.array("user#" + str(size)).repeat(size)
    for i in number[:,0]:
        path[count] = "←".join(data[number[count,1:(i+2)]])
        user[count] = "user#"+str(count+1)
        count += 1
    return path,user

In [3]:
%load_ext cython

In [4]:
%%cython -a
# 使用cython優化
cimport numpy as np
import numpy as np
def data_generate_c(int size = 10000,int leng = 11):
    cdef np.ndarray data = np.array([chr(i) for i in range(65,65 + leng -1)])
    cdef np.ndarray number = np.random.randint(10, size = (size,leng))
    cdef int count = 0
    cdef np.ndarray path = np.array("←".join(data)).repeat(size)
    cdef np.ndarray user = np.array("user#" + str(size)).repeat(size)
    for i in number[:,0]:
        path[count] = "←".join(data[number[count,1:(i+2)]])
        user[count] = "user#"+str(count+1)
        count += 1
    return path,user

In [5]:
# 相鄰不重複資料產生
# 隨機產生一個值為 0~9 相鄰不重複的陣列
def data_generate(range_ = 9):
    import random
    arr = np.ones(1)
    long = random.randint(2, 10)
    for i in range(long):
        b = random.randint(0, range_)
        if (arr[len(arr)-1] == b):
            i -= 1
        else:
            arr = np.append(arr,b)
    return arr
def data_merge(size = 10000, leng = 11):
    import numpy as np
    import random
    data = np.array([chr(i) for i in range(65,65 + leng -1)])
    count = 0
    path = np.array("←".join(data)).repeat(size)
    user = np.array("user#" + str(size)).repeat(size)
    for i in range(size):
        temp = np.round(data_generate()).astype(int)
        path[count] = np.array("←".join(data[temp[1:]]))
        if path[count] == "":
            path[count] = data[random.randint(0, 9)]      
        count += 1
    return path, user

In [5]:
%timeit data_generate(1000000,11)
%timeit data_generate_c(1000000,11)

8.48 s ± 326 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.38 s ± 965 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit data_generate(10000,11)
%timeit data_generate_c(10000,11)
# 使用cython版本提升了10%左右的效率

59.4 ms ± 2.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
55.4 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
path,user = data_merge(100000,11)
df = pd.DataFrame(path,columns = ["PATH"],index = user)

# 方法一
## 直接使用groupby進行計算

In [7]:
# 添加計數
df["quantity"] = 1
df.head()

,PATH,quantity
user#100000,A←D←I←B←H←B←E←B,1
user#100000,E←D←A←B←A,1
user#100000,F←B←H,1
user#100000,H←D←H←G←F←B←F,1
user#100000,I←A←C←I←E←I←C←H←I←G,1


In [8]:
%timeit df.groupby("PATH").count().sort_values(by = "quantity",ascending = False)

145 ms ± 5.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
method1 = df.groupby("PATH").count().sort_values(by = "quantity",ascending = False)

In [10]:
method1[method1.index.str.len() == 9].head()

,quantity
PATH,
G←D←C←F←H,4
A←I←E←J←H,4
D←B←C←J←C,3
H←D←C←B←H,3
D←B←C←D←C,3


## 方法一缺點
1.資料量龐大的情況下,最為耗時(10萬筆數據 145 ms)<br/>

# 方法二
## 切分資料之後再進行groupby

In [11]:
df.head()

,PATH,quantity
user#100000,A←D←I←B←H←B←E←B,1
user#100000,E←D←A←B←A,1
user#100000,F←B←H,1
user#100000,H←D←H←G←F←B←F,1
user#100000,I←A←C←I←E←I←C←H←I←G,1


In [12]:
df_2 = df["PATH"].str.split('←',expand = True)

In [13]:
df_2 = df_2.fillna("")
df_2.head()

,0,1,2,3,4,5,6,7,8,9
user#100000,A,D,I,B,H,B,E,B,,
user#100000,E,D,A,B,A,,,,,
user#100000,F,B,H,,,,,,,
user#100000,H,D,H,G,F,B,F,,,
user#100000,I,A,C,I,E,I,C,H,I,G


In [14]:
df_2["quantity"] = 1
df_2.head()

,0,1,2,3,4,5,6,7,8,9,quantity
user#100000,A,D,I,B,H,B,E,B,,,1
user#100000,E,D,A,B,A,,,,,,1
user#100000,F,B,H,,,,,,,,1
user#100000,H,D,H,G,F,B,F,,,,1
user#100000,I,A,C,I,E,I,C,H,I,G,1


In [15]:
# 篩選指定長度
df_len_5 = df_2[(df_2[5] != "") & (df_2[6] == "")]
df_len_5.head()

# 切分並還原字串
df_3 = pd.DataFrame(df_len_5[0] + "←" + df_len_5[1] + "←" + df_len_5[2] + "←" + df_len_5[3] + "←" + df_len_5[4] + "←" + df_len_5[5],columns = ["PATH"])
df_3["quantity"] = 1
df_3.head()

,PATH,quantity
user#100000,G←F←I←G←C←H,1
user#100000,F←A←F←I←G←C,1
user#100000,H←G←H←D←E←C,1
user#100000,H←I←G←B←F←E,1
user#100000,H←E←F←B←E←G,1


In [16]:
# 另一種較迅速的方法
df_4 = df[(df['PATH'].str.len()>=5) & (df['PATH'].str.len() < 6)]

In [17]:
%timeit df_3.groupby("PATH").count()

17 ms ± 689 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 方法二缺點
1.操作繁瑣(使用df_4的方法較簡單)<br/>
2.當資料量龐大時仍然緩慢
### 優於方法一的部分在於事先切分資料,計算量較小